<a href="https://colab.research.google.com/github/japarra27/backend-application/blob/master/convert_img.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import os
import datetime
import tempfile
from shutil import move
from PIL import ImageDraw, ImageFont, Image
from google.cloud import storage

In [31]:
storage_client = storage.Client.from_service_account_json('dsc-projects.json')
files = storage_client.list_blobs('dsc-files', prefix='designs_library/processing')

In [32]:
files = [img.name for img in files]

In [34]:
def convert_upload_img(original_image, author, im_height=800, im_width=600):
  with Image.open(original_image) as image:
    image = image.resize((im_height, im_width))
    draw = ImageDraw.Draw(image)
    font = ImageFont.truetype('Roboto-Bold.ttf', size=16)
    designer_name = author
    image_name = original_image.split("/")[-1]
    date = datetime.datetime.now().strftime("%Y-%m-%d")
    (x, y) = (int(image.size[0]*0.35), int(image.size[1]*0.95))
    message = f"{designer_name} - {date}"
    color = 'rgb(0, 0, 0)'
    draw.text((x, y), message, fill=color, font=font)
    image.save(original_image, format="png")

In [35]:
for img in files:
  try:
    file_name = storage_client.bucket('dsc-files').get_blob(img)
    image_name = img.split("/")[-1]
    _, temp_local_filename = tempfile.mkstemp()
    file_name.download_to_filename(temp_local_filename)
    convert_upload_img(original_image=temp_local_filename, author='andrew')
    new_blob = storage_client.bucket('dsc-files').blob(f'designs_library/converted/{image_name}')
    new_blob.upload_from_filename(temp_local_filename, content_type='image/png')
    source_bucket = storage_client.bucket('dsc-files')
    source_blob = source_bucket.blob(img)
    source_bucket.copy_blob(source_blob, source_bucket, f'designs_library/source/{img.split("/")[-1]}')
    source_blob.delete()
  except:
    pass